In [1]:
from ipywidgets import HBox, VBox, IntSlider, interactive, SelectionSlider, Label, Layout
from IPython.display import display
import folium
import json
import pandas as pd
import geopandas as gpd
import geopatra
import random

In [6]:
def test(passenger_count,private_bus_to_public_bus,travel_risk,confirmed_risk,qtn_risk):
    district_data_df = pd.read_csv("district_info.csv")
    numbers_df = district_data_df
    district_data_df["AtRisk"] = (
        numbers_df["BusCount"] 
        * passenger_count
        * private_bus_to_public_bus
        / passenger_count
        * private_bus_to_public_bus
        * travel_risk
        + numbers_df["QTNCount"] / qtn_risk
        + confirmed_risk * numbers_df["CaseCount"]
    ).astype(int)
    
    geo_df = gpd.read_file("ka.json")
    df = gpd.GeoDataFrame(pd.merge(district_data_df, geo_df, on="district"), geometry='geometry')
    display(df.folium.chloropeth(color_by="AtRisk", color="red",style = {'color': 'black',
                    'weight': 1,
                    'dashArray': '10, 5',
                    'fillOpacity': 0.5,
                  }, zoom=6, tiles='googlehybrid', tooltip=['district','AtRisk'], location=(15.317277, 75.713890)))

In [23]:
slider_layout = Layout(width='500px',height='20px')
style_dict = {'description_width': '150px'}
float_value = [10**-i for i in range(1,11)]
float_selections = [("%g"%i,i) for i in float_value]


a = interactive(
    test,
    passenger_count = IntSlider(
        value=50, 
        min=0, 
        max=100,
        description="Passenger Count: ",
        style = style_dict,
        layout = slider_layout
    ),
    
    private_bus_to_public_bus = IntSlider(
        value=50, 
        min=0, 
        max=100,
        description="Private bus to Public bus: ",
        style = style_dict,
        layout = slider_layout
    ),
    
    confirmed_risk = IntSlider(
        value=50, 
        min=0, 
        max=100,
        description="Confirmed Risk: ",
        style = style_dict,
        layout = slider_layout
    ),
    
    travel_risk = SelectionSlider(
        options=float_selections,
        value=float_value[len(float_selections)//2],
        description="Travel Risk: ",
        style = style_dict,
        layout = slider_layout
    ),
    
    qtn_risk = SelectionSlider(
        options=float_selections,
        value=float_value[len(float_selections)//2],
        description="QTN Risk: ",
        style = style_dict,
        layout = slider_layout
    ),
)

In [24]:
display(a)

interactive(children=(IntSlider(value=50, description='Passenger Count: ', layout=Layout(height='20px', width=…